<a href="https://scipp.github.io"><img src="https://scipp.github.io/_static/logo-2022.svg" width="600" /></a>

# Multi-dimensional arrays with labeled dimensions and physical units

## [scipp.github.io](https://scipp.github.io)

<br>
<br><br>

<table style="margin-left:0px;">
    <tr>
        <td>
            <h1>About me</h1><br>
            <h2>Neil Vaytet</h2>
            <h3>
            <ul>
                <li>Scientific software developer @ <a href="https://europeanspallationsource.se/">European Spallation Source (DK/SE)</a></li>
                <li>Python for scientific data analysis</li>
                <li>Data visualization</li>
            </ul>
            </h3>
        </td>
        <td>
            <img src="neil.png" width="200" /> &nbsp;
            <img src="https://europeanspallationsource.se/themes/custom/ess/logo.svg" width="300" />
        </td>
    </tr>
</table>


<h3>
    <img src="simon.png" width="60" /> Simon Heybrock &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
    <img src="janlukas.png" width="60" />Jan-Lukas Wynen &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
    <img src="sunyoung.png" width="60" />Sunyoung Yoo
</h3>

<br><br><br><br>
<br><br><br><br>
<br><br><br><br>

## Outline

### 1. Introduction to labeled dimensions
### 2. Going further: physical units, bin-edge coordinates
### 3. Binned data: multi-dimensional handling of event/record data
### 4. Plopp: building interactive visualizations


<br><br><br><br>
<br><br><br><br>
<br><br><br><br>
<br><br><br><br>

## 1. Introduction to labeled dimensions

Why do we need labeled dimensions?

In [ ]:
%matplotlib inline
import numpy as np
import scipp as sc
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=1234)

In [ ]:
def plot(*x):
    """
    Useful plot function for 1d and 2d data
    """
    fig, ax = plt.subplots()
    for a in x:
        if a.ndim == 1:
            ax.plot(np.arange(len(a)), a)
        elif a.ndim == 2:
            ax.imshow(a, origin="lower")

def scatter(x, y):
    """
    Simple scatter plot
    """
    fig, ax = plt.subplots()
    ax.scatter(x, y, marker=".", s=1)
    ax.set_aspect("equal")
    ax.set_xlim(x.min(), x.max())
    ax.set_ylim(y.min(), y.max())
    return ax

In [ ]:
ny, nx = 10, 20
a = np.sin(np.arange(ny) / (ny / 4)).reshape((-1, 1)) * np.cos(np.arange(nx) / (ny / 4))
a.shape

In [ ]:
plot(a)

In [ ]:
# Slice out row number 4
plot(a[4, :])

### We can't always deduce from the shape

When both dimensions have the same length,
it can sometimes be difficult to remember which dimension is which:

In [ ]:
ny, nx = 20, 20
a = np.sin(np.arange(ny) / (ny / 4)).reshape((-1, 1)) * np.cos(np.arange(nx) / (ny / 4))
a.shape

In [ ]:
plot(a)

In [ ]:
plot(a[:, 7], a[7, :])

### The situation gets worse with more dimensions

Say I now have an array that has 4 dimensions: `x, y, z, time` (in that order if I'm lucky?).

In [ ]:
a = np.random.random([20] * 4)
a.shape

I want to get the first `z` slice...

Which one was it again?

In [ ]:
z_slice = a[:, :, 0, :]  # x,y,z,t
z_slice = a[0, :, :, :]  # z,y,x,t
z_slice = a[:, :, :, 0]  # t,x,y,z

<br><br><br><br><br><br><br><br><br>


### Introducing labeled dimensions

<img src="https://docs.xarray.dev/en/stable/_static/dataset-diagram-logo.png" width="220" /> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; <img src="https://scipp.github.io/_static/logo-2022.svg" width="220" />

[Xarray](https://docs.xarray.dev/en/stable/index.html) (https://docs.xarray.dev) introduced labels to multi-dimensional Numpy arrays.

"*real-world datasets are usually more than just raw numbers; they have labels which encode information about how the array values map to locations in space, time, etc.*"

We have embraced, and to a large extent copied, the Xarray mechanism.

In [ ]:
var = sc.array(dims=["x", "y", "z", "time"], values=a)
var

Getting the `z` slice is now easy and **readable**

In [ ]:
var["z", 0]

### Adding coordinates

Coordinates can be specified for each dimension.
Essentially, they describe the extent of each axis, as well as how far each data point is from its neighbours.

In [ ]:
data = sc.array(dims=["latitude", "longitude"], values=rng.random((5, 9)))
sc.show(data)

In Scipp and Xarray, coordinates are added in a data structure called `DataArray`:

In [ ]:
da = sc.DataArray(
    data=data,
    coords={
        "longitude": sc.linspace("longitude", -120, 120, 9),
        "latitude": sc.linspace("latitude", -70, 70, 5),
    },
)
da

In [ ]:
sc.show(da)

In [ ]:
da.plot()

### Automatic broadcasting

Because of the labeled dimensions,
operations between arrays with different dimensions can automatically broadcast operands to the correct shape:

In [ ]:
a = sc.array(dims=["y", "x"], values=rng.random((50, 50)))  # 2D array
b = sc.array(dims=["y"], values=np.arange(50.))  # 1D array
c = a * b
c

In [ ]:
c.plot(aspect='equal')

There is no longer a need for Numpy's `x.reshape(50, 1)`!

<br><br><br><br><br><br><br><br>
<br><br><br><br><br><br><br><br>

## 2. Going further

<img src="https://scipp.github.io/_static/logo-2022.svg" width="220" />

### 2.1 Physical units

Every data variable and coordinate in Scipp has physical units.

In [ ]:
s = rng.normal(size=(2, 10000))
h = np.histogram2d(s[0], s[1], bins=(100, 100))[0]

image = sc.array(dims=["y", "x"], values=h, unit="counts")
image

In [ ]:
image.plot(aspect="equal")

In [ ]:
integration_time = sc.scalar(300.0, unit="s")
image /= integration_time
print(image.unit)

image.plot(aspect="equal")

### Units also provide protection

Say I now have a background image (dark frame) which I want to subtract from the signal image above,
but I forgot to first normalize it by integration time

In [ ]:
background = sc.array(dims=["y", "x"], values=rng.random((100, 100)), unit="counts")

image - background

In [ ]:
background_integration_time = sc.scalar(60.0, unit="s")
background /= background_integration_time

image - background

The units are very useful in early prevention of difficult-to-spot bugs in a workflow.

They save **hours** of debugging time, free-up mental capacity and let the user focus on the important thing: **doing science**.

<br><br>

(see also [pint](https://pint.readthedocs.io/en/stable/), [astropy.units](https://docs.astropy.org/en/stable/units/index.html), [pint-xarray](https://pint-xarray.readthedocs.io/en/stable/), ...)

<br><br><br><br><br><br><br><br>
<br><br><br><br><br><br><br><br>

### 2.2 Bin-edge coordinates

It is sometimes necessary to have coordinates that represent a range for each data value.
E.g. "the atmospheric pressure was 1000 hPa between latitudes of 30 and 35 degrees".

Scipp supports this by having **bin-edge coordinates**: a coordinate which has a length of 1 more than the dimension length.

In [ ]:
pressure = sc.DataArray(
    data=sc.array(dims=["latitude", "longitude"], values=1000. + rng.random((5, 9)), unit="hPa"),
    coords={
        "latitude": sc.linspace("latitude", -90, 90, 6),
        "longitude": sc.linspace("longitude", -180, 180, 10),
    },
)
sc.show(pressure)

In [ ]:
pressure

This also arises every time we histogram data:

In [ ]:
v = sc.array(dims=["event"], values=rng.normal(loc=300.0, size=10000), unit="K")

# Histogram and plot the data
v.hist(temperature=50).plot()

<br><br><br><br><br><br><br><br><br><br>
<br><br><br><br><br><br><br><br><br><br>

## 3. Binned data

Scipp distinguishes **histogrammed** data from **binned** data:

- Histogrammed data refers to regular dense arrays of, e.g., floating-point values with an associated bin-edge coordinate.
- Binned data refers to the precursor of histogrammed data, i.e., each bin contains a “list” of contributing events or values. Binned data can be converted into a histogram by computing the sum over all events or values in a bin.

This is conceptually similar to a multi-dimensional [AwkwardArray](https://awkward-array.org/doc/main/).

<br><br><br><br><br><br><br><br><br><br>
<br><br><br><br><br><br><br><br><br><br>
<br><br><br><br><br><br><br><br><br><br>

It is best illustrated with an example of data analysis.
For this, we will use one of the NYC taxi datasets.

<img src="https://vaex.readthedocs.io/en/latest/_images/datasets_2_1.png" /> <img src="https://cdn-images-1.medium.com/v2/resize:fit:2680/1*fqrY2h4uLD3eKEvJ6hlI2g.png" width="600" />

(https://vaex.readthedocs.io/en/latest/datasets.html, Dataset from 2015, obtained as a HDF5 file from the Vaex docs,
and subsequently cleaned of outliers).

In [ ]:
%matplotlib widget

da = sc.io.load_hdf5('nyc_taxi_data_2015.h5')
da

In [ ]:
n = 1000
x = da.coords["longitude"].values[::n]
y = da.coords["latitude"].values[::n]
scatter(x, y)

### Binning the data records

Working with binned data is most efficient when keeping the number of bins relatively low.

Binning is essentially like overlaying a grid of bin edges onto our data:

In [ ]:
ax = scatter(x, y)
for lon in np.linspace(*ax.get_xlim(), 9):
    ax.axvline(lon, color="gray")
for lat in np.linspace(*ax.get_ylim(), 9):
    ax.axhline(lat, color="gray")

In [ ]:
# Bin into 8 longitude & latitude bins
binned = da.bin(latitude=8, longitude=8)
binned

In [ ]:
binned.hist().plot(aspect="equal", norm="log")

### Selecting/slicing bins

Binning groups the data into bins, but keeps the underlying table beneath; **no information is lost, it is simply re-ordered**.
The bins can then be used for slicing the data, providing extremely efficient data selection and filtering.

For example, we select one bin in Manhattan by slicing both `longitude` and `latitude` dimensions:

In [ ]:
manh = binned["longitude", 1]["latitude", 4]
manh

Because the underlying data is still available, we can histogram this with a much finer resolution:

In [ ]:
manh.hist(latitude=300, longitude=300).plot(norm="log", aspect="equal")

We select another bin, which contains the JFK airport:

In [ ]:
jfk = binned["longitude", 6]["latitude", 1]
jfk.hist(latitude=300, longitude=300).plot(norm="log", aspect="equal")

![jfk](https://upload.wikimedia.org/wikipedia/commons/thumb/5/5a/JFK_airport_terminal_map.png/640px-JFK_airport_terminal_map.png)

(https://commons.wikimedia.org/wiki/File:JFK_airport_terminal_map.png)

### Binning into a new dimension

Data that has already been binned can be binned further into new dimensions, because the underlying records from the original table are still available.

In [ ]:
manh

In the following, we look at the trip distances inside the Manhattan and JFK bins we have selected above.

In [ ]:
# Use 100 distance bins
manh_dist = manh.bin(trip_distance=100)
manh_dist

In [ ]:
manh_dist.hist().plot()

In [ ]:
jfk_dist = jfk.bin(trip_distance=100)
jfk_dist.hist().plot()

### Other operations on bins: what is the mean fare amount as a function of distance?

In addition to summing/histogramming, bins can be used for other reduction operations: `min()`, `max()`, and `mean()`.

To illustrate this, we will now inspect a new variable in our Manhattan data which is the fare amount (in dollars).

We start from our result from the previous section, where the Manhattan data has been binned into 100 `'trip_distance'` bins.

In [ ]:
manh_dist

We use the `.bins` property to access the underlying coordinate values of the points that lie inside our selected map area.
We can then look at the properties of those coordinates.

For example, to get the minimum and maximum fares for all trips that ended inside our Manhattan area, we can do

In [ ]:
manh_dist.bins.coords['fare_amount'].min(), manh.bins.coords['fare_amount'].max()

These values are somewhat strange, indicative of bad data in the table.

To proceed further in our analysis, we shall restrict our fare range from \\$0 to \\$200.

We first want to visually inspect the fare amount as a function of trip distance.

In [ ]:
# Make 100 bins between 0 and 200 dollars
nbins = 100
fare_bins = sc.linspace('fare_amount', 0, 200, nbins + 1, unit='dollar')

# Bin & plot our data
manh_dist.bin(fare_amount=fare_bins).hist().transpose().plot(norm="log")

Some things we can say about the data:

- there appears to be a (somewhat expected) correlation between fare amount and trip distance: the further you go, the more you'll have to pay
- for a given trip distance, clients usually pay above the diagonal line, but very rarely below
- there appears to be a magic fare amount (~\\$52) that will take you anywhere from 0 to 60 miles (will come back to this later)

Our goal is now to try and compute some average fare amount as a function of distance.

We again use the `.bins` property to get to the `'fare_amount'` coordinate, showing it is made up of 100 bins in the `'trip_distance'` dimension:

In [ ]:
manh_dist.bins.coords['fare_amount']

We can compute the mean fare inside each `'trip_distance'` bin using

In [ ]:
mean_fare = manh_dist.bins.coords['fare_amount'].bins.mean()
mean_fare

This is *almost* what we were after, except that it contains only values.
We need to combine this with the coordinate of the `'trip_distance'` bins:

In [ ]:
# Remember to add the coordinate for the `trip_distance` bins back
mean_fare = sc.DataArray(data=mean_fare,
                         coords={'trip_distance': manh_dist.coords['trip_distance']})
mean_fare.plot()

### Filtering out the magic \\$52 fare

We would like to clean up our `fare_amount` vs `trip_distance` relation by filtering out all trips that have a fare amount of \\$52.

One way to do this would be to use Numpy masking or smart indexing to filter out all \\$52 fares in the original data table.
But this can potentially be quite a costly operation (both in CPU and memory, as the list of indices to save could be large).

An alternative way is to once again use bins.

We make 3 bins in the `'fare_amount'` dimension, where the middle bin is very narrow, centered around \\$52.

In [ ]:
# Make 3 bins = 4 bin edges
fare_bins = sc.array(dims=['fare_amount'], values=[0, 51.75, 52.25, 200], unit='dollar')
manh_dist_fare = manh_dist.bin(fare_amount=fare_bins)
manh_dist_fare

Once we have this, we leave the middle bin out by indexing with a step of 2,
concatenate the first and last `'fare_amount'` bins into a single bin using `concat()`,
and finally compute the mean fare as we did above.

In [ ]:
#                                   Access fare_amount coord | Select first & last bin | Concatenate        | Compute mean as above
mean_fare_filtered = manh_dist_fare.bins.coords['fare_amount']['fare_amount', ::2].bins.concat('fare_amount').bins.mean()
mean_fare_filtered

In [ ]:
# Remember to add the coordinate for the `trip_distance` bins back
mean_fare_filtered = sc.DataArray(data=mean_fare_filtered,
                                  coords={'trip_distance': manh_dist.coords['trip_distance']})

# Plot both results
import plopp as pp
pp.plot({'unfiltered': mean_fare, 'filtered': mean_fare_filtered})

We can now see that the \\$52 fares were introducing significant skew in our result.

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>

## 4. Plopp: building interactive visualizations

<img src="https://scipp.github.io/plopp/_static/logo.svg" width="600" />

https://scipp.github.io/plopp

In [ ]:
import plopp as pp
from plopp import widgets
import ipywidgets as ipw
from scipp.scipy.ndimage import gaussian_filter

In [ ]:
data = da.group("hour").hist(latitude=500, longitude=500)
data

### Goal: make an interactive visualization with 3 panels and a slider

![plopp_visu](plopp.png)

In [ ]:
# Slider node that provides index to slice
slider = ipw.IntSlider(description="Hour:", min=0, max=23)
slider_node = pp.widget_node(slider)

# Node that actually does the slicing
slice_node = pp.Node(lambda da, ind: da["hour", ind], da=data, ind=slider_node)

# Add a 2D figure to show the NYC map
fig2d = pp.figure2d(slice_node, norm="log", cbar=False)

# Add a node after the slicing to sum along the latitude dimension
sum_lat = pp.Node(sc.sum, slice_node, dim="latitude")
# Add a node after the slicing to sum along the longitude dimension
sum_lon = pp.Node(sc.sum, slice_node, dim="longitude")

# Add a node after the sum that performs as Gaussian smoothing
smooth = pp.Node(gaussian_filter, sum_lat, sigma=5)

# Add a 1D figure that will display sum along longitude
fig_lon = pp.figure1d(sum_lon, norm="log")
# Add a 1D figure that will display both raw latitude sum and smoothed data
fig_lat = pp.figure1d(sum_lat, smooth, norm="log")

widgets.Box([slider, [fig2d, fig_lon], fig_lat])  # Container box

In [ ]:
pp.show_graph(fig_lat)

### Adding a second widget for the Gaussian smoothing kernel size

In [ ]:
# Slider node that provides index to slice
slider = ipw.IntSlider(description="Hour:", min=0, max=23)
slider_node = pp.widget_node(slider)

# Node that actually does the slicing
slice_node = pp.Node(lambda da, ind: da["hour", ind], da=data, ind=slider_node)

# Add a 2D figure to show the NYC map
fig2d = pp.figure2d(slice_node, norm="log", cbar=False)

# Add a node after the slicing to sum along the latitude dimension
sum_lat = pp.Node(sc.sum, slice_node, dim="dropoff_latitude")
# Add a node after the slicing to sum along the longitude dimension
sum_lon = pp.Node(sc.sum, slice_node, dim="dropoff_longitude")


# Add a new slider that will act as input to the Gaussian smoothing node
smooth_slider = ipw.IntSlider(description="kernel:", min=1, max=25)
smooth_slider_node = pp.widget_node(smooth_slider)


# Add a node after the sum that performs as Gaussian smoothing
smooth = pp.Node(gaussian_filter, sum_lat, sigma=smooth_slider_node)

# Add a 1D figure that will display sum along longitude
fig_lon = pp.figure1d(sum_lon, norm="log")
# Add a 1D figure that will display both raw sum and smoothed data
fig_lat = pp.figure1d(sum_lat, smooth, norm="log")

widgets.Box([[slider, smooth_slider], [fig2d, fig_lon], fig_lat])  # Container box

In [ ]:
pp.show_graph(fig_lat)